# API

> Module to get data

In [1]:
#| default_exp api

In [1]:
# | exporti

import datetime
from datetime import datetime as dt
import pandas as pd
import altair as alt
from altair_data_server import data_server
alt.data_transformers.enable('data_server')

from entsoe import EntsoePandasClient
from ents import db


In [2]:
def plot_heatmap(df):
    chart = alt.Chart(df).mark_area().encode(
    x="Timestamp:T",
    y="Generation:Q",
    color="Type:N",
    tooltip="Type:N"
)
    chart.display()

In [3]:
# | export
def process_df(df):
    df = df.drop('Actual Consumption', axis=1, level=1)
    df.reset_index(inplace=True, names = 'Timestamp')
    df = df.droplevel(1, axis=1)
    df = df.melt('Timestamp', var_name='Type', value_name='Generation')
    return df

In [4]:
#| hide
# https://github.com/EnergieID/entsoe-py
# https://transparency.entsoe.eu/content/static_content/Static%20content/web%20api/Guide.html#_generation_domain
conn = db.create_connection('test.db')
token = open('../token', 'r').read()
client = EntsoePandasClient(api_key=token)

start = pd.Timestamp('20230101', tz='Europe/Brussels')
end = start + pd.Timedelta(days=1)
country_code = 'NL'  # Belgium
df = client.query_generation(country_code, start=start,end=end, psr_type=None, nett=False)

df = process_df(df)
plot_heatmap(df)
df.to_sql('test2.db', conn, if_exists='append')

d:\Programmeren\adventofcode\.aocenvpy311\Lib\site-packages\altair\utils\core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

760